In [2]:
!pip install dagshub


  Using cached dacite-1.6.0-py3-none-any.whl.metadata (14 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
Using cached dacite-1.6.0-py3-none-any.whl (12 kB)
Using cached tenacity-9.0.0-py3-none-any.whl (28 kB)
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 38.4 MB/s eta 0:00:0000:01
  Attempting uninstall: graphql-core
    Found existing installation: graphql-core 3.2.6
    Uninstalling graphql-core-3.2.6:
      Successfully uninstalled graphql-core-3.2.6


In [17]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# Now you can import from src
from src.data_utils import load_and_process_taxi_data

import pandas as pd

import dagshub
dagshub.init(repo_owner="siddharthadhikari85", repo_name="NYC_taxi", mlflow=True)

Initialized MLflow to track repo "siddharthadhikari85/NYC_taxi"

Repository siddharthadhikari85/NYC_taxi initialized!

In [18]:
rides1 = load_and_process_taxi_data(year=2022)
rides2=(load_and_process_taxi_data(year=2023))
rides = pd.concat([rides1,rides2],ignore_index=True)

File already exists for 2022-01.
Loading data for 2022-01...
Total records: 2,463,931
Valid records: 2,415,141
Records dropped: 48,790 (1.98%)
Successfully processed data for 2022-01.
File already exists for 2022-02.
Loading data for 2022-02...
Total records: 2,979,431
Valid records: 2,921,118
Records dropped: 58,313 (1.96%)
Successfully processed data for 2022-02.
File already exists for 2022-03.
Loading data for 2022-03...
Total records: 3,627,882
Valid records: 3,551,986
Records dropped: 75,896 (2.09%)
Successfully processed data for 2022-03.
File already exists for 2022-04.
Loading data for 2022-04...
Total records: 3,599,920
Valid records: 3,522,113
Records dropped: 77,807 (2.16%)
Successfully processed data for 2022-04.
File already exists for 2022-05.
Loading data for 2022-05...
Total records: 3,588,295
Valid records: 3,509,056
Records dropped: 79,239 (2.21%)
Successfully processed data for 2022-05.
File already exists for 2022-06.
Loading data for 2022-06...
Total records: 3,55

In [19]:
temp_rides = rides[rides["pickup_location_id"] == 43]

In [20]:
from src.data_utils import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(temp_rides)
ts_data.head()

/Users/siddharthadhikari/github/NYC_taxi/src/data_utils.py:223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rides["pickup_hour"] = rides["pickup_datetime"].dt.floor("h")


,pickup_hour,pickup_location_id,rides
0,2022-01-01 00:00:00,43,96
1,2022-01-01 01:00:00,43,60
2,2022-01-01 02:00:00,43,22
3,2022-01-01 03:00:00,43,8
4,2022-01-01 04:00:00,43,4


In [21]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2022-01-01 00:00:00,43,96
1,2022-01-01 01:00:00,43,60
2,2022-01-01 02:00:00,43,22
3,2022-01-01 03:00:00,43,8
4,2022-01-01 04:00:00,43,4
...,...,...,...
17515,2023-12-31 19:00:00,43,55
17516,2023-12-31 20:00:00,43,72
17517,2023-12-31 21:00:00,43,50
17518,2023-12-31 22:00:00,43,28


In [22]:
ts_data = ts_data.drop(columns=["pickup_location_id"])

In [23]:
print(ts_data.head())


          pickup_hour  rides
0 2022-01-01 00:00:00     96
1 2022-01-01 01:00:00     60
2 2022-01-01 02:00:00     22
3 2022-01-01 03:00:00      8
4 2022-01-01 04:00:00      4


In [25]:
!pip install pmdarima
from pmdarima import auto_arima

# Find best ARIMA(p, d, q)
model_auto = auto_arima(ts_data["rides"], seasonal=False, stepwise=True, trace=True)

# Print the best parameters
print(model_auto.summary())


Performing stepwise search to minimize aic


/opt/anaconda3/envs/NYC_taxi/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/NYC_taxi/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/NYC_taxi/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/NYC_taxi/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/NYC_taxi/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure

 ARIMA(2,0,2)(0,0,0)[0]             : AIC=162977.313, Time=0.96 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=210122.120, Time=0.05 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=165205.279, Time=0.12 sec


/opt/anaconda3/envs/NYC_taxi/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/NYC_taxi/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/NYC_taxi/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(0,0,1)(0,0,0)[0]             : AIC=191455.692, Time=0.42 sec


...

 ARIMA(3,0,5)(0,0,0)[0]             : AIC=158948.964, Time=5.55 sec


/opt/anaconda3/envs/NYC_taxi/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(5,0,5)(0,0,0)[0]             : AIC=inf, Time=5.81 sec


/opt/anaconda3/envs/NYC_taxi/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(4,0,4)(0,0,0)[0] intercept   : AIC=158569.328, Time=11.41 sec

Best model:  ARIMA(4,0,4)(0,0,0)[0]          
Total fit time: 61.705 seconds
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                17520
Model:               SARIMAX(4, 0, 4)   Log Likelihood              -78411.047
Date:                Wed, 05 Mar 2025   AIC                         156840.094
Time:                        00:32:38   BIC                         156910.034
Sample:                             0   HQIC                        156863.123
                              - 17520                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          3.1061      0.023    136.709      0.000       3.

In [26]:
from statsmodels.tsa.arima.model import ARIMA

# Fit the best ARIMA model (Example: ARIMA(4, 0, 4) if suggested)
model_arima = ARIMA(ts_data["rides"], order=(4, 0, 4))  # Replace with best (p, d, q)
arima_result = model_arima.fit()

# Print model summary
print(arima_result.summary())

/opt/anaconda3/envs/NYC_taxi/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                               SARIMAX Results                                
Dep. Variable:                  rides   No. Observations:                17520
Model:                 ARIMA(4, 0, 4)   Log Likelihood              -78001.908
Date:                Wed, 05 Mar 2025   AIC                         156023.816
Time:                        00:36:20   BIC                         156101.527
Sample:                             0   HQIC                        156049.404
                              - 17520                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         73.4150      0.768     95.634      0.000      71.910      74.920
ar.L1          1.6885      0.007    251.108      0.000       1.675       1.702
ar.L2          0.2262      0.006     35.274      0.0

In [27]:
from sklearn.metrics import mean_absolute_error

# Split data into train (80%) and test (20%)
train_size = int(len(ts_data) * 0.8)
train, test = ts_data.iloc[:train_size], ts_data.iloc[train_size:]

# Fit ARIMA on training data
model_arima = ARIMA(train["rides"], order=(4, 0, 4))  # Use optimal (p,d,q)
arima_result = model_arima.fit()

# Forecast for test set
forecast = arima_result.forecast(steps=len(test))

# Compute MAE
mae = mean_absolute_error(test["rides"], forecast)
print("Mean Absolute Error (MAE) for ARIMA:", mae)


/opt/anaconda3/envs/NYC_taxi/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Mean Absolute Error (MAE) for ARIMA: 24.95946505301096


In [28]:
import mlflow
mlflow.set_experiment("model_arima")

# Start an MLflow run
with mlflow.start_run():
        mlflow.log_metric("mean_absolute_error", mae)

2025/03/05 00:39:39 INFO mlflow.tracking.fluent: Experiment with name 'model_arima' does not exist. Creating a new experiment.


🏃 View run delightful-toad-963 at: https://dagshub.com/siddharthadhikari85/NYC_taxi.mlflow/#/experiments/6/runs/83013bba38a341f99e99b058d5ce1888
🧪 View experiment at: https://dagshub.com/siddharthadhikari85/NYC_taxi.mlflow/#/experiments/6
